# TO DO:

## Что уже есть:

# Сохраняем модель в виде словаря: где ключ -- слово, значение -- вектор, ему соответствующий.

В дальнейшем, если решим использовать первую модель, этот код будет изменен, поэтому для обработки строки файла не будем создавать отдельную функцию, а обработаем все на месте.

In [2]:
#Открываем файл
glove_model = open("glove.txt", "r") 

Для удобства работы с данными напишем функцию, которая будет преобразовывать вектор в числовой.

In [3]:
def list_to_float(list_char):
    """
    Все элементы списка из текстовых делает числовыми
    :type list_char: list
    :param list_char: список с текстовыми элементами
    :return: list числовой
    """
    list_float = []
    for elem in list_char:
        list_float.append(float(elem))
    return list_float

Построчно считываем и сразу записываем в словарь.

In [4]:
model = {}
stopwords = ['.', ',', ':', '"', '?', '!', '<', '>', '(', ')', '@', '#', '$', '^', '*', '/', '_']
for line in glove_model:
    splitted_line = line.split(' ')
    key = splitted_line.pop(0)
    if key not in stopwords:
        splitted_line = list_to_float(splitted_line)
        model[key] = splitted_line
line = glove_model.readline()

# Теперь считываем сам текст из файла.

In [5]:
f = open('input.txt', 'r')

Определим функцию для разбивания строк на токены.

In [38]:
stoplist = ['.', ',', ':', '"', '?', '!', '<', '>', '(', ')', '@', '#', '$', '^', '*', '/', '\n', ' ', '-', '_']
def tokenize(string, tokens):
    """
    Разбивает входную строку на токены
    :type string: str
    :param string: строка, которая будет разбита на токены
    :type tokens: list
    :param tokens: список токенов
    """
    current_word = str('')
    for char in string:
        if char.isalpha() and char not in stoplist:
            current_word += char
            continue
        else:
            if len(current_word) > 0:
                if current_word.lower() not in tokens and current_word not in stoplist:
                    tokens.append(current_word.lower())
            if char.isalpha() and char not in stoplist:
                current_word = char
            else:
                current_word = str('')

    if len(current_word) > 0:
        if current_word.lower() not in tokens and current_word not in stoplist:
            tokens.append(current_word.lower())

In [39]:
tokens = []

for line in f:
    tokenize(line, tokens)

Для каждого токена, который есть в модели, получаем из данных и сохраняем его вектор.

In [40]:
vectors = {}
    
for token in tokens:
    if token in model:
        vectors[token] = model[token]

In [41]:
#Отладочный вывод
#for key in vectors:
#    print key, ':', vectors[key]
#    print '\n'

Определим функцию, суммирующую векторы всех слов абзаца. Сразу воспользуемся алгоритмом Кахана, чтобы уменьшить погрешность при суммировании.



In [19]:
def paragraph_vec(paragraph):
    """
    Подсчитывает сумму векторов слов абзаца
    :type paragraph: list of strings
    :param paragraph: список всех слов абзаца
    :return: list 300-vector
    """
    par_vec = []
    for x in xrange(300):
        res = 0.0
        error = 0.0
        for word in paragraph:
            if word in model:
                y = model[word][x] - error
                t = res + y
                error = (t - res) - y
                res = t
        par_vec.append(res)
    return par_vec

In [ ]:
import numpy as np
def par2vecwithNP (paragraph):
    res = np.array([0.0]*300)
    error = np.array([0.0]*300)
    for word in paragraph:
        if word in model:
            y = np.array(model[word]) - error
            t = res + y
            error = (t - res) - y
            res = t
    return res


In [28]:
import time
res1 = []
res2 = []
st = time.time()
for i in xrange(10):
    res1.append(paragraph_vec(genPar[i]))
print time.time()-st

st = time.time()
for i in xrange(10):
    res2.append(par2vecwithNP(genPar[i]))
print time.time()-st

0.0764248371124
0.0120670795441


Для текущей модели скорее всего вполне подойдет и простое суммирование в лоб, но на всякий случай для подстраховки оставим еще функцию скалярного произведения, где используется алгоритм Кахана, даже если в итоге она не понадобится.

In [31]:
def scalar_prod(vec1, vec2):
    """
    Подсчитывает скалярное произведение двух векторов
    :type vec1: list
    :param vec1: 300-вектор первого слова
    :type vec2: list
    :param vec2: 300-вектор второго слова
    :return: float
    """
    return np.dot(vec1, vec2)

12355.329097948295

In [32]:
def scalar_prod_kahan(vec1, vec2):
    """
    Подсчитывает скалярное произведение двух векторов
    :type vec1: list
    :param vec1: 300-вектор первого слова
    :type vec2: list
    :param vec2: 300-вектор второго слова
    :return: float
    """
    res = 0.0
    error = 0.0
    for x in xrange(300):
        mult = vec1[x]*vec2[x]
        y = mult - error
        t = res + y
        error = (t - res) - y
        res = t
    return res

12355.329097948295

In [33]:
print scalar_prod(model['hello'], model['world'])

3.20338782697


In [34]:
print scalar_prod_kahan(model['hello'], model['world'])

3.20338782697


In [35]:
from math import sqrt
def norm(vec):
    return sqrt(scalar_prod(vec, vec))

In [45]:
def cosine(vec1, vec2):
    len1 = norm(vec1)
    len2 = norm(vec2)
    if len1 == 0 or len2 == 0:
        return 0
    else:
        cos = scalar_prod(vec1, vec2)/(len1*len2)
    return cos

In [46]:
Paragraph1 = 'A single Kingdom of Great Britain resulted from the Union of Scotland and England (which already comprised the present-day countries of England and Wales) in 1707'
par1 = []
tokenize(Paragraph1, par1)

In [47]:
Paragraph2 = 'Great Britain refers geographically to the island of Great Britain, politically to England, Scotland and Wales in combination.[27] However, it is sometimes used loosely to refer to the whole of the United Kingdom'
par2 = []
tokenize(Paragraph2, par2)

Взяли два произовольных абзаца, для каждого нашли его суммарный вектор. Посмотрим на косинусное между ними.

In [48]:
par1_vec = paragraph_vec(par1)
par2_vec = paragraph_vec(par2)
cos = cosine(par1_vec, par2_vec)
print cos

0.91373987894


Теперь для каждого слова первого абзаца вычисляем косинусное расстояние с каждым словом второго абзаца, берём минимум; эти числа усредняем для всех слов первого абзаца

In [55]:
def max_cosines(text1, text2):
    """
    Подсчитывает максимальное косинусное расстоение между словом первого абзаца и всеми словами второго.
    Затем усредняет все эти значения.
    :type text1: list
    :param text1: список всех слов первого текста
    :type text2: list
    :param text2: список всех слов второго текста
    :return: float
    """
    max_cosines = [max([cosine(model[w1], model[w2]) for w2 in text2 if w2 in model]) for w1 in text1 if w1 in model] 
    av = sum(max_cosines)/len(max_cosines)
    return av

In [56]:
print max_cosines(par1, par2)

0.74766992635


Считываем датасет, который устроен так: в каждой строчке три абзаца, первые два взяты из одной книжки, третий из другой. "Правильный" ответ всегда на втором месте, "неправильный" на третьем.
Всего строчек 10000.

In [57]:
dataset = open("gutenberg_task.txt", 'r')

In [58]:
def split_line(line):
    """
    Разделяет текст по знакам табуляции и сохраняет в список
    :type line: string
    :param line: текст
    :return: list
    """
    sents = []
    first_sym_num = 0
    last_sym_num = 0
    for symbol in line:
        last_sym_num += 1
        if symbol == '\t':
            word = line[first_sym_num:last_sym_num]
            sents.append(word)
            first_sym_num = last_sym_num
    word = line[first_sym_num:]
    sents.append(word)
    return sents

In [59]:
text = []
for line in dataset:
    text.append(split_line(line))

In [ ]:
res1 = []
res2 = []
i = 0
for paragraph in text:
    if i % 50 == 0:
        print i
    sent1 = paragraph[0]
    sent2 = paragraph[1]
    sent3 = paragraph[2]
    sent1_list = []
    sent2_list = []
    sent3_list = []
    tokenize(sent1, sent1_list)
    tokenize(sent2, sent2_list)
    tokenize(sent3, sent3_list)
    sent1_vec = paragraph_vec(sent1_list)
    sent2_vec = paragraph_vec(sent2_list)
    sent3_vec = paragraph_vec(sent3_list)
    res1_par = [] 
    res2_par = []
    res1_par.append(cosine(sent1_vec, sent2_vec))
    res1_par.append(cosine(sent1_vec, sent3_vec))
    res1.append(res1_par)
    res2_par.append(max_cosines(sent1_list, sent2_list))
    res2_par.append(max_cosines(sent1_list, sent3_list))
    res2.append(res2_par)
    i += 1

Подсчитаем точности алгоритмов и их доверительные интервалы.

In [87]:
l1 = np.array(map(lambda x: int(x[0]>x[1]), res1))
l2 = np.array(map(lambda x: int(x[0]>x[1]), res2))

In [96]:
#Доверительные интервалы уровня доверия 0,95. нужная квантиль - 1,96
m1 = l1.mean()
m2 = l2.mean()
d1 = 1.96*sqrt(m1*(1-m1)/len(l1))
d2 = 1.96*sqrt(m2*(1-m2)/len(l2))
print "Точность первого алгоритма:", m1
print "Точность второго алгоритма:", m2
print "Доверительный интервал первого алгоритма:", (m1-d1, m1+d1)
print "Доверительный интервал второго алгоритма:", (m2-d2, m2+d2)

Точность первого алгоритма: 0.7356
Точность второго алгоритма: 0.666
Доверительный интервал первого алгоритма: (0.72695614133720365, 0.74424385866279641)
Доверительный интервал второго алгоритма: (0.65675586201747294, 0.67524413798252714)


Получается, что первый алгоритм обладает большей точностью.